In [1]:
import torch
from torch import nn

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect") if down else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        X = self.conv(x)
        return self.dropout(X) if self.use_dropout else X

In [2]:
Block(10, 100, down=False, use_dropout=False)

Block(
  (conv): Sequential(
    (0): ConvTranspose2d(10, 100, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [3]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()

        self.initial = nn.Sequential(
            nn.Conv2d(in_channels*2, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2)
        ) # 128

        self.down1 = Block(features, features*2, down=True, act="relu", use_dropout=False) # 64
        self.down2 = Block(features*2, features*4, down=True, act="relu", use_dropout=False) # 32
        self.down3 = Block(features*4, features*8, down=True, act="relu", use_dropout=False) # 16
        self.down4 = Block(features*8, features*8, down=True, act="relu", use_dropout=False) # 8
        self.down5 = Block(features*8, features*8, down=True, act="relu", use_dropout=False) # 4
        self.down6 = Block(features*8, features*8, down=True, act="relu", use_dropout=False) # 2

        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, padding_mode="reflect"),
            nn.ReLU() # 1
        )

        self.up1 = Block(features*8, features*8, down=False, act="relu", use_dropout=False) # 2
        self.up2 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=False) # 4
        self.up3 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=False) # 8
        self.up4 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=False) # 16
        self.up5 = Block(features*8*2, features*4, down=False, act="relu", use_dropout=False) # 32
        self.up6 = Block(features*4*2, features*2, down=False, act="relu", use_dropout=False) # 64
        self.up7 = Block(features*2*2, features, down=False, act="relu", use_dropout=False) # 128

        self.finalup = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, x, z):
        d1 = self.initial(torch.cat([x,z], 1))
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)

        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        return self.finalup(torch.cat([up7, d1], 1))


In [4]:
g = Generator(in_channels=3, features=64)

s1 = torch.rand([1,3,256,256])
s2 = torch.rand([1,3,256,256])

g(s1, s2).shape

torch.Size([1, 3, 256, 256])

In [5]:
s1

tensor([[[[0.2068, 0.4127, 0.0040,  ..., 0.2035, 0.6573, 0.6828],
          [0.5773, 0.7272, 0.3109,  ..., 0.8190, 0.2375, 0.8926],
          [0.3870, 0.5245, 0.3777,  ..., 0.9491, 0.2352, 0.6513],
          ...,
          [0.5627, 0.3154, 0.1969,  ..., 0.4791, 0.5171, 0.2437],
          [0.1703, 0.4178, 0.0495,  ..., 0.0049, 0.3487, 0.3534],
          [0.2256, 0.7917, 0.2395,  ..., 0.4870, 0.2894, 0.8672]],

         [[0.9384, 0.1058, 0.7233,  ..., 0.9545, 0.8682, 0.5022],
          [0.6541, 0.9720, 0.7170,  ..., 0.4267, 0.7089, 0.1788],
          [0.9076, 0.2569, 0.5120,  ..., 0.2696, 0.3614, 0.8534],
          ...,
          [0.2922, 0.8430, 0.5647,  ..., 0.4054, 0.8479, 0.5501],
          [0.5948, 0.0053, 0.1732,  ..., 0.8298, 0.3789, 0.4631],
          [0.9477, 0.6178, 0.7103,  ..., 0.3031, 0.5350, 0.6743]],

         [[0.9722, 0.9651, 0.1220,  ..., 0.8206, 0.1902, 0.7015],
          [0.6982, 0.0756, 0.1067,  ..., 0.3490, 0.9218, 0.0101],
          [0.3268, 0.3667, 0.2249,  ..., 0